In [ ]:
import torch
import torch.optim as optim
from build_dataset import IMDBDataset, build_dataloader
from build_vocab import transform
from fc_net import FcNet
from tqdm import tqdm

In [12]:
imdb_dataset_train = IMDBDataset(is_train=True)
train_loader = build_dataloader(imdb_dataset_train, batch_size=10)

In [14]:
print('词表长度:', len(imdb_dataset_train.vocab))
print('句子长度/seq length:', imdb_dataset_train.max_length)

词表长度: 147158
句子长度/seq length: 2752


In [15]:
batch_size = 10
seq_len = imdb_dataset_train.max_length
epochs = 2

In [18]:
net = FcNet()
loss = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())
for epoch in range(epochs):
    bar = tqdm(train_loader, total=len(train_loader))
    for X, y in bar:
        optimizer.zero_grad()
        
        y_hat = net(X)
        l = loss(y_hat, y)
        l.backward()
        optimizer.step()
        bar.set_description(f'epoch {epoch}, loss: {l}')
torch.save(net.state_dict(), './saved_model/fc.params')

/Users/chenbai/Projects/ml/imdb-sentiment/fc_net.py:24: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self.softmax(out)
epoch 2, loss: 1.8801422119140625:  41%|████      | 1019/2500 [36:54<53:38,  2.17s/it]  


KeyboardInterrupt: 

In [ ]:
# test
def test(model, imdb_dataset):
    loss = torch.nn.CrossEntropyLoss()
    test_loss = 0
    correct = 0
    model.eval()
    test_dataloader = build_dataloader(imdb_dataset,is_train=False)
    with torch.no_grad():
        for X, y in tqdm(test_dataloader):
            y_hat = model(data)
            l = loss(y_hat, y)
            pred = y_hat.max(1, keepdim=True)[1]  # 获取最大值的位置,[batch_size,1]
            correct += pred.eq(y.view_as(pred)).sum()   # 预测正确的个数
    test_loss /= len(test_dataloader.dataset)
    print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_dataloader.dataset),
        100. * correct / len(test_dataloader.dataset)))
test()

In [ ]:
def predict():
    comment = '''I don't believe anyone does western films better than Kevin Costner and with Taylor Sheridan writing it is a win win for viewers. Take a look at "Wind River" for an example of what Taylor Sheridan can do. Where has Cole Hauser been hiding? He is amazing. I guess the daughter is a necessary part, but she is over the top at being a bitch. The scenery is fantastic and being from Colorado I can really appreciate the vast beautiful land. Do yourself a favor and get hooked on this series as it is going to be a winner.'''
    x = transform(text, imdb_dataset_train.vocab)
    # [b, seq]
    x.reshape(1, -1)
    y_hat = net(x)
    pred = y_hat.max(1, keepdim=True)[1].view(1)
    print(pred)
    